In [1]:
%matplotlib widget

import cv2 as cv
import numpy as np
import yaml
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
uploader = widgets.FileUpload(accept='.yml', multiple=False)
display(uploader)

In [ ]:
calib, = yaml.safe_load(uploader.value[0].content.tobytes())
camera = calib['camera']
focus = camera['focus']
sx = camera['sx']
sy = camera['sy']
cx = camera['cx']
cy = camera['cy']
k1 = camera['k1']
k2 = camera['k2']
k3 = camera['k3']
p1 = camera['p1']
p2 = camera['p2']
image_width = camera['image_width']
image_height = camera['image_height']

observ_points = [np.array(p, np.float32) for p in calib['calib_obj']['observ_points']]
object_points = [np.array(p, np.float32) for p in calib['calib_obj']['object_points']]
# reproject_points = [np.array(p) for p in calib['calib_obj']['reproject_points']]
# rvecs = np.array(calib['calib_obj']['rvecs'])
# tvecs = np.array(calib['calib_obj']['tvecs'])
# diff_points = [observ_point - reproject_point for observ_point, reproject_point in zip(observ_points, reproject_points)]

npos = len(object_points)
# ylim_min, ylim_max = np.min(diff_points), np.max(diff_points)
# xlim_max = np.max([p.shape[0] for p in object_points])

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(object_points, observ_points, imageSize=(image_width, image_height), cameraMatrix=None, distCoeffs=None)